In [1]:
source("lib/NelderMead.R")

In [2]:
data(iris)
dat = iris
dat$Sepal.LenWidRatio = dat[,"Sepal.Length"] / dat[,"Sepal.Width"]
dat$Petal.LenWidRatio = dat[,"Petal.Length"] / dat[,"Petal.Width"]
dat$SepalPetal.Length = dat[,"Sepal.Width"] + dat[,"Petal.Length"]
dat$SepalPetal.Width = dat[,"Petal.Length"]/dat[,"Petal.Width"]
dat$SepalPetal.LenWidRatio = dat[,"SepalPetal.Length"]/dat[,"SepalPetal.Width"]
dat$Len.SepPetRatio = dat[,"Sepal.Length"]/dat[,"Petal.Length"]
dat$Width.SepPetRatio = dat[,"Sepal.Width"]/dat[,"Petal.Width"]
dat$SepLenPetWidRatio = dat[,"Sepal.Length"]/dat[,"Petal.Width"]
dat$SepWidPetLenRatio = dat[,"Sepal.Width"]/dat[,"Petal.Length"]
library("Matrix")
dat_mod = model.matrix(Species~.-1, data = dat, sparse = TRUE)
dat_sparse = sparse.model.matrix(Species~.-1, data = dat)
dat_y = as.numeric(dat[,"Species"])-1

## Example using xgboost

### [1] Creating a function to wrap the algo and obj function you wish to optimize on for input into NM_opt()

In [3]:
library(xgboost)

In [4]:
# If you are using xgboost version from March 2017 onwards
xgb_wrap_obj <- function(param_fixed, vtx, dat_x, dat_y, nfolds = 5, pred = FALSE, verb = FALSE ){ 
  return(xgb.cv(param=param_fixed, nround = vtx["nround"],
                max_depth = vtx["max_depth"], eta = vtx["eta"],
                data= dat_x, label = dat_y, nfold = nfolds,
                stratified = TRUE, prediction=pred, verbose=verb, missing = NA)$evaluation_log$test_mlogloss_mean[vtx["nround"]] )
            # ***** For previous version of xgboost, sometime before June 2016 ******
            #    - replace the last line with the following
            #    - stratified = TRUE, prediction=pred, verbose=verb, missing = NA)$test.mlogloss.mean[vtx["nround"]] )
}
#---------------------------------------------------------------------------------------------------
# Points to note here:
#  [1] NM_opt() is built to MINIMIZE the objective function, for objective functions which requires
#      maximization, simply add minus (-) to the objective value
#  [2] The first 2 inputs name, "param_fixed" and "vtx", cannot be changed
#         - "param_fixed" is the static parameters input to the predictive model used.
#         - "vtx" is used to hold the variable parameters the the model is to be optimized on
#         - in this example we are optimizing on xgboost's "nround", "max_depth" and "eta"
#---------------------------------------------------------------------------------------------------

### [2] If any of the parameters are unconstrained, a boundry function is need for inverse mapping

In [5]:
xgb_bdry <- function(vtx = vector(0), ind_int = character(0), ind_num = character(0), min_int = integer(0)) {
  vtx[ind_int[1]] <- exp(vtx[ind_int[1]])
  vtx[ind_int[2]] <- exp(vtx[ind_int[2]])
  #to ensure that integer outputs are in their appropriate integer format
  vtx[ind_num] <- exp(vtx[ind_num])/(1+exp(vtx[ind_num]))
  vtx[ind_int][ vtx[ind_int] < min_int ] <- min_int
  vtx[ind_int] <- round(vtx[ind_int])
  return(vtx)
}
#---------------------------------------------------------------------------------------------------
# Points to note here:
#  [1] The only compulsory input for a boundary function is that the first input, "vtx = vector(0)",
#      must reference the variable parameters.
#  [2] The subsequent inputs can be built according to the user's design preference but will be
#      based on the following principles:
#             i. all constrained parameters must be included in this function
#            ii. to enable NM_opt to use these functions to map the vertices back to their
#                constrained values for input into the predictive model to compute the obj value
#---------------------------------------------------------------------------------------------------

### [3] Other inputs into the NM_opt() function

In [6]:
xgb_bdry_params <- list( ind_int = c("nround","max_depth"), ind_num = c("eta"), min_int = 1)
#---------------------------------------------------------------------------------------------------
# Points to note here:
#   [1] based on the boundary function constructed, these are the input parameters into 'xgb_bdry'.
#         i. typically it would include the index of the parameters
#        ii. and parameter bounds
#---------------------------------------------------------------------------------------------------

In [7]:
initial_vertices <- cbind(nround = log(c(100,200,300,50)), 
                                  max_depth = log(c(10,3,6,20)), 
                                  eta = -log(1/c(0.2, 0.05, 0.09, 0.005) - 1) )
#---------------------------------------------------------------------------------------------------
# Points to note here:
#   [1] Initialization of algorithm paramters.
#         - as Nelder Mead is a simplex optimization algorithm, we need P+1 sets of parameters
#           to initialize the optimization
#         - e.g., if we are optimization on P = 3 parameters, we will need 4 inputs for each parameter
#           as is the case for this example
#   [2] Inputs should be mapped to (-inf, +inf) numerical range
#         - this is important as Nelder Mead is an UNCONSTRAINED optimization algorithm
#         - this is also why we need a boundary function to map the unconstrained parameters
#           back to their orignal from for input into the predictive algorithm
#---------------------------------------------------------------------------------------------------

In [8]:
model_input <- list(param = list("nthread" = 20,   # number of threads to be used 
                               "objective" = "multi:softmax",    # binary classification 
                               "eval_metric" ="mlogloss",    # evaluation metric
                               "num_class" = 3
                                ),
                  dat_x = dat_sparse,
                  dat_y = dat_y )
#---------------------------------------------------------------------------------------------------
# Points to note here:
#  [1] The inputs here should match those found in the wrapper function for the selected model
#             - in this case it refers to "xgb_wrap_obj"
#  [2] The subsequent inputs can be built according to the user's design preference but will be
#      based on the following principles:
#             i. all constrained parameters must be included in this function
#            ii. to enable NM_opt to use these functions to map the vertices back to their
#                constrained values for input into the predictive model to compute the obj value
# [3] do.call() function is used to pass these parameters to 'obj_fun', hence the input format
#     has to be a list.
#---------------------------------------------------------------------------------------------------

### Executing the NM algo on xgboost

In [9]:
vtcs <- NM_opt( vtcs_init = initial_vertices,
                obj_fun = xgb_wrap_obj,
                fxd_obj_param  = model_input,
                bdry_fun = xgb_bdry,
                fxd_bdry_param = xgb_bdry_params,
                max_0prgrss = 50,
                a_e = 2.5
)
#---------------------------------------------------------------------------------------------------
# NM_opt() input arguments
# vtcs_init,            # initialiation of Nelder Mead vertices (to note that have to be in the range (-inf, +inf)
# obj_fun,              # objective function
# fxd_obj_param,        # objective function fixed parameter inputs
# bdry_fun = NULL,      # boundary function to map vtcs for input into the selected algorithm
# fxd_bdry_param = NULL,# parameters for input into the boundary functions
# max_iter = 200,       # max iterations the algorithm will make in the search for local/global minima
# max_0prgrss = 10,     # max iterations the algorithm will from the last improvement in objective value
# vtcs_tol = 3,         # decimal places vertices tolerance before vtcs are considered equivalent
# a_r = 1, a_e = 2, a_c = 0.5, a_s = 0.5 # Nelder Mead parameters
# *** Description of Nelder Mead parameters ***
#  a_r -> reflect_cstnt
#  a_e -> expand_cstnt
#  a_c -> ctrt_cstnt
#  a_s -> shrink_cstnt
#-------------------------------------------------------------------------------------

[1] "-----------------------------------------------------------"
[1] "   Current iteration: 1"
[1] "-----------------------------------------------------------"
[1] "Objective delta: 0.1306812"
[1] "Nelder Mead step taken: contraction"
[1] "Current vertices coordinates and objective value"
           obj nround max_depth       eta
[1,] 0.1306812     21         2 0.6063184
[2,] 0.1592780    300         6 0.0900000
[3,] 0.1777320    100        10 0.2000000
[4,] 0.2059932    200         3 0.0500000
[1] "-----------------------------------------------------------"
[1] ""
[1] "-----------------------------------------------------------"
[1] "   Current iteration: 2"
[1] "-----------------------------------------------------------"
[1] "Objective delta: 0"
[1] "Nelder Mead step taken: contraction"
[1] "Current vertices coordinates and objective value"
           obj nround max_depth       eta
[1,] 0.1306812     21         2 0.6063184
[2,] 0.1411256    104         5 0.1174402
[3,] 0.1592780 

## Example using randomForest

In [10]:
library(caret)
K_Folds_K = 3
K_Folds = createFolds(dat_y, k = K_Folds_K, list = TRUE, returnTrain = FALSE)

library(doParallel)
cl <- makeCluster(K_Folds_K-1)
registerDoParallel(cl)

Loading required package: lattice
Loading required package: ggplot2
Loading required package: foreach
Loading required package: iterators
Loading required package: parallel


### [1] Wrapper over randomForest algo and customized cross validation calculation

In [11]:
rf_wrap_ll_par <- function(vtx, ipt_target, ipt_data, idx_folds , ipt_ntrees = 100, ipt_replace = TRUE, ipt_do.trace = FALSE )	{
  k_folds <- length(idx_folds)
  ll_ttl <- foreach(i_cv = 1:k_folds, .combine=sum, .packages=c("randomForest")) %dopar% {
    N_i <- length(ipt_target) - length(idx_folds[[i_cv]])
    rf <- randomForest( ipt_target[-idx_folds[[1]]]~., data = ipt_data[-idx_folds[[1]],], importance=FALSE,
                        ntrees = vtx["ntrees"], replace = ipt_replace, mtry = vtx["mtry"], do.trace = ipt_do.trace )
    prob1 <- matrix(predict(rf, ipt_data[-idx_folds[[1]],],type = "prob", predict.all=TRUE)$aggregate, ncol=3)
    return( -sum(log(prob1[matrix(as.logical(model.matrix( ~ipt_target[-idx_folds[[1]]]-1, sparse = TRUE)),ncol=3)]), na.rm = TRUE) / N_i)
  }
  return(ll_ttl/k_folds)
}

### [2] Boundary function wrapper for the selected randomForest parameters

In [12]:
rf_bdry <- function(vtx = vector(0), ind_int = character(0), ind_num = character(0), min_int = integer(0)) {
  vtx[ind_int[1]] <- exp(vtx[ind_int[1]])
  vtx[ind_int][ vtx[ind_int] < min_int ] <- min_int
  vtx[ind_int] <- round(vtx[ind_int])
  #to ensure that integer outputs are in their appropriate integer format
  vtx[ind_num] <- exp(vtx[ind_num])/(1+exp(vtx[ind_num]))
  return(vtx)
}

### [3] Other inputs into the NM_opt() function

In [13]:
initial_vertices <- cbind(ntrees = log(c(50,200,400)),
                          mtry = log(c(7,20,5)))

In [14]:
rf_bdry_params <- list( ind_int = c("ntrees", "mtry"), min_int = 1)

In [15]:
model_input <- list(ipt_target = factor(dat_y),
                    ipt_data = dat_mod,
                    idx_folds = K_Folds)

### Executing the NM algo on randomForest

In [16]:
vtcs0 <- NM_opt( vtcs_init = initial_vertices,
                 obj_fun = rf_wrap_ll_par,
                 fxd_obj_param  = list( ipt_target = factor(dat_y),
                                        ipt_data = dat_mod,
                                        idx_folds = K_Folds),
                 bdry_fun = rf_bdry,
                 fxd_bdry_param = list( ind_int = c("ntrees", "mtry"), min_int = 1),
                 a_e = 2.5,
                 max_0prgrss = 30)

[1] "-----------------------------------------------------------"
[1] "   Current iteration: 1"
[1] "-----------------------------------------------------------"
[1] "Objective delta: 0.0320497820989167"
[1] "Nelder Mead step taken: contraction"
[1] "Current vertices coordinates and objective value"
            obj ntrees mtry
[1,] 0.03204978    200    3
[2,] 0.03238083    142    2
[3,] 0.03273110    400    2
[1] "-----------------------------------------------------------"
[1] ""
[1] "-----------------------------------------------------------"
[1] "   Current iteration: 2"
[1] "-----------------------------------------------------------"
[1] "Objective delta: 0"
[1] "Nelder Mead step taken: shrinking"
[1] "Current vertices coordinates and objective value"
            obj ntrees mtry
[1,] 0.03204978    200    3
[2,] 0.03303338    169    2
[3,] 0.03329085    283    2
[1] "-----------------------------------------------------------"
[1] ""
[1] "------------------------------------------